In [319]:
import utils
import numpy as np

In [324]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
        A region growing algorithm that segments an image into 1 or 0 (True or False).
        Finds candidate pixels with a Moore-neighborhood (8-connectedness). 
        Uses pixel intensity thresholding with the threshold T as the homogeneity criteria.
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            seed_points: list of list containing seed points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            T: integer value defining the threshold to used for the homogeneity criteria.
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    ### START YOUR CODE HERE ### (You can change anything inside this block)
    # You can also define other helper functions
    ###############Array Delete same seed points previously traversed############################
    def delete_traversed(im, a, b):
        #a is new seed points and b is already traversed seed points
        #we also check valid seed_points that does not go beyond boundrary values
        z = []
        for idx in range(len(a)):
            unique = 0
            for idy in range(len(b)):
                if (a[idx][0]== b[idy][0] and a[idx][1]== b[idy][1]):
                    unique += 1
            if unique==0 and (-1<a[idx][0]<im.shape[0] and -1<a[idx][1]<im.shape[1]):
                z.append([a[idx][0],a[idx][1]])
        z = np.asarray(z)
        return z
    ###############################################################################
    #######################Generate New Seed Points##########################################
    def gen_seed_points(im, seed_points, thres):
        new_seed_points = []
        x = 0
        for idx in range(len(seed_points)):
            #valid seed points
            if (0<seed_points[idx][0]<im.shape[0] and 0<seed_points[idx][1]<im.shape[1]):
                h = seed_points[idx][0]
                w = seed_points[idx][1]
                #coordinate for pixel color of first seed point
                hc = 254
                wc = 138
                #If neighbour pixels meet the threshold criteria
                if abs(im[hc,wc]-im[h-1,w-1])<thres:
                    new_seed_points.append([h-1,w-1])
                    x+=1
                if abs(im[hc,wc]-im[h-1,w])<thres:
                    new_seed_points.append([h-1,w])
                    x+=1
                if abs(im[hc,wc]-im[h-1,w+1])<thres:
                    new_seed_points.append([h-1,w+1])
                    x+=1
                if abs(im[hc,wc]-im[h,w-1])<thres:
                    new_seed_points.append([h,w-1])
                    x+=1
                if abs(im[hc,wc]-im[h,w+1])<thres:
                    new_seed_points.append([h,w+1])
                    x+=1
                if abs(im[hc,wc]-im[h+1,w-1])<thres:
                    new_seed_points.append([h+1,w-1])
                    x+=1
                if abs(im[hc,wc]-im[h+1,w])<thres:
                    new_seed_points.append([h+1,w])
                    x+=1
                if abs(im[hc,wc]-im[h+1,w+1])<thres:
                    new_seed_points.append([h+1,w+1])
                    x+=1
        new_seed_points = np.asarray(new_seed_points)
        #remove duplicate seed points if any
        new_seed_points = np.unique(new_seed_points, axis = 0)

        return new_seed_points
    #########################################################################################
    threshold = T
    traversed_seed_points = seed_points
    x = 0
    while True:
        x = x+1
        #Updating seed Points that has been traversed and met the condition
        if x!=0:
            traversed_seed_points = np.append(traversed_seed_points, seed_points, axis=0)
        #Generating new seed points that met the criteria
        new_seed_points = gen_seed_points(im, seed_points, threshold)
        #Deleting already traversed seed points from newly generated seed points
        new_seed_points = delete_traversed(im, new_seed_points, traversed_seed_points)
        #if there is no new seed points we break the loop
        if new_seed_points.shape[0]==0:
            break
        #Updating current seed points to newly found seed points
        seed_points = new_seed_points
    #Segmentation based on all traversed seed points
    segmented = np.zeros_like(im).astype(bool)
    im = im.astype(float)
    for row, col in traversed_seed_points:
        segmented[row, col] = True
    return segmented
    ### END YOUR CODE HERE ###

In [327]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)

Reading image: images/defective-weld.png
Saving image to: image_processed/defective-weld-segmented.png


/tmp/ipykernel_7668/4171674125.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
